# LangChain Expression Language (LCEL)

- 기본 예시 : 프롬프트 + 모델 + 출력 파서
    - 가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것이다.

## 기본 설정

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

os.environ['LANGSMITH_PROJECT'] = 'CH01-Basic'
os.environ['HF_HOME'] = '../' # model weight 저장 위치를 현재 폴더로 수정

## 프롬프트 템플릿의 활용

- `PromptTemplate`
    - 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿 입니다.

- 사용법
    - `template` : 템플릿 문자열이다. 이 문자열 내에서 중괄호 `{}` 는 변수를 나타낸다.
    - `input_variables` : 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

In [3]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTmeplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

## Chain 생성

- LCEL 을 사용하여 다양한 구성 요소를 단일 체인으로 결합한다.
- 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달한다.

In [16]:
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")

### Model 설정

- HuggingFace 내에 있는 `DeepSeek-R1-Distill-Qwen-1.5B` 모델을 사용하도록 하겠다.

In [18]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

### Chain 설정

In [23]:
import torch

def model_generate(prompt):
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
    encode_token = tokenizer(prompt.text, return_tensors='pt', padding=True, truncation=True)
    model.to(device)
    with torch.no_grad():
        output = model.generate(
            input_ids = encode_token['input_ids'].to(device),
            attention_mask = encode_token['attention_mask'].to(device),
            temperature = 0.6,
            max_new_tokens = 50
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text


In [24]:
chain = prompt | model_generate

## `invoke()` 호출

- python 딕셔너리 형태로 입력값을 전달한다.
- `invoke()` 함수 호출 시 입력값을 전달합니다.

In [25]:
input = {'topic' : 'Principles of AI Model Training'}

In [26]:
chain.invoke(input)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


"Please explain Principles of AI Model Training in simple terms. I need to understand this for my project. Please explain step by step.\n\nOkay, so I need to explain the principles of AI model training in simple terms. I know a little about AI, but I'm not an expert. Let me start by"

### Stream 출력

In [28]:
answer = chain.stream(input)

for chunk in answer:
    print(chunk, end='', flush=True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Please explain Principles of AI Model Training in simple terms. Need to explain in a way that someone with no technical background can understand. Also, need to explain why each principle is important. Maybe some examples would help.

Also, I have to make sure that the explanation is clear and concise, avoiding jargon

## 출력 파서 (Output Parser)

In [29]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model_generate | output_parser

In [30]:
input = {'topic' : 'Principle of AI model Training'}

chain.invoke(input)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


"Please explain Principle of AI model Training in simple terms. I need to understand it for my AI project.\nOkay, so I have a project where I'm trying to build an AI model that can predict stock prices. I'm a bit confused about how AI models are trained. I remember from school that there"

### stream 출력

In [31]:
answer = chain.stream(input)

for chunk in answer:
    print(chunk , end='', flush=True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Please explain Principle of AI model Training in simple terms. What are the key factors that determine the success of an AI model training?
Please explain Principle of AI model Training in simple terms. What are the key factors that determine the success of an AI model training?
</think>

**AI Model Training Explained Simply

## 템플릿을 변경하여 적용



In [5]:
import torch

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM

template = """
You are a 10-year experienced Korean teacher. Please write an Korean conversation based on the given situation in the [FORMAT] below.

Situation:
{question}

FORMAT:
	•	Korean Conversation:
	•	English Translation:
"""

prompt = PromptTemplate.from_template(template)

## model
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

## output-parser
output_parser = StrOutputParser()

## Chain

def model_generate(prompt):
    device = 'mps' if torch.backends.mps.is_available() else 'cpu'
    encode_token = tokenizer(prompt.text, return_tensors='pt', padding=True, truncation=True)
    model.to(device)

    input_length = encode_token['input_ids'].shape[1]

    with torch.no_grad():
        output = model.generate(
            input_ids = encode_token['input_ids'].to(device),
            attention_mask = encode_token['attention_mask'].to(device),
            temperature = 0.6,
            max_new_tokens = 100
        )
    
    generated_ids = output[0][input_length:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    return generated_text

chain = prompt | model_generate | output_parser

answer = chain.stream({'question' : 'I want to go to a restaurant and order food.'})

for chunk in answer:
    print(chunk, end='', flush=True)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


	•	•	Question:
	•	•	Answer:

Please ensure that your answers are in Korean and English.
Alright, so I want to go to a restaurant and order food. First, I need to figure out the best time to get there. Maybe it's around lunchtime or early evening. I should probably book a table in advance because there might be competition. Once I have a seat, I can start ordering dishes. I'll make a list of what I want, like